In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz --user
nlp = en_core_web_sm.load()
import pandas as pd
import json
!pip install bs4 --user
from bs4 import BeautifulSoup
import requests
import re
import os
import mmap

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data

In [2]:
with open('data.json', 'r') as f:
    dict_json_weighted = json.load(f)
object_1 = list(dict_json_weighted.keys())[4]
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])

0-I'm looking for a 4 star hotel. 
1-I've found 21 4 star hotels. Is there a certain area you want to stay in?
2-Yes, in the east, please. 
3-I have found several hotels. Do you have any preference towards price?
4-I would not mind finding one with free wifi.
5-Unfortunately, we don't have any hotels with those preference in the area, but we do have 6 guesthouses available. Would a guesthouse be okay?
6-A guesthouse is fine. Which one would you recommend?
7-I recommend the Leverton House. It's a cheap guesthouse, but still a 4-star establishment.
8-That sounds good.  Can you give me the address and postcode for Leverton House?
9-They are located at 732-734 newmarket road, and postcode is cb58rs
10-Thank you. I'm also looking for a venetian restaurant on the east side as well. 
11-I'm sorry there are no Venetian restaurants. Are there any cuisines I can look for?
12-Yes, how about one that serves Indian food?
13-There are four restaurants that serve indian food in the east. Do you have 

## Load ontology

In [3]:
with open('ontology.json', 'r') as o:
    ontology = json.load(o)
action=['inform', 'request', 'select', 'recommend', 'not found', 'request booking', 'offer booking', 
        'inform booked', 'decline booking', 'welcome','greet', 'bye', 'reqmore' ]
topic= ['hotel', 'restaurant', 'taxi', 'train', 'attraction', 'police', 'hospital']

## display topics and actions

In [4]:
top=[]
act=[]
ont=list(ontology.keys())
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        if str(x).lower() in topic:
            if str(x).lower() not in top:
                top.append(str(x).lower()) 
        if x.lemma_ in action:
            if x.lemma_ not in act:
                act.append(x.lemma_)
                 
print("topics: "+str(top))
print("actions: "+str(act))


I'm looking for a 4 star hotel. 
I've found 21 4 star hotels. Is there a certain area you want to stay in?
Yes, in the east, please. 
I have found several hotels. Do you have any preference towards price?
I would not mind finding one with free wifi.
Unfortunately, we don't have any hotels with those preference in the area, but we do have 6 guesthouses available. Would a guesthouse be okay?
A guesthouse is fine. Which one would you recommend?
I recommend the Leverton House. It's a cheap guesthouse, but still a 4-star establishment.
That sounds good.  Can you give me the address and postcode for Leverton House?
They are located at 732-734 newmarket road, and postcode is cb58rs
Thank you. I'm also looking for a venetian restaurant on the east side as well. 
I'm sorry there are no Venetian restaurants. Are there any cuisines I can look for?
Yes, how about one that serves Indian food?
There are four restaurants that serve indian food in the east. Do you have a price range?
Book one of them 

## Display the updated slots 

In [13]:
for i in dict_json_weighted[object_1]['goal']:
    if i!='message':
        if dict_json_weighted[object_1]['goal'][i] !={}:
            print("****** "+i+" ******")
            for j in (dict_json_weighted[object_1]['goal'][i]):
                if dict_json_weighted[object_1]['goal'][i][j] not in ['fail_info', {}]:
                    print("°"+j)
                    print(dict_json_weighted[object_1]['goal'][i][j])
    else:
        print("****** "+i+" ******")
        print(dict_json_weighted[object_1]['goal'][i])

****** taxi ******
°reqt
['car type', 'phone']
°info
{'arriveBy': '14:15'}
****** hotel ******
°reqt
['postcode', 'address', 'type']
°info
{'stars': '4', 'internet': 'yes', 'area': 'east'}
°fail_info
{'stars': '4', 'internet': 'no', 'area': 'east'}
****** message ******
['You are planning your trip in Cambridge', "You are looking for a <span class='emphasis'>place to stay</span>. The hotel <span class='emphasis'>doesn't need to include internet</span> and should have <span class='emphasis'>a star of 4</span>", "The hotel should be in the <span class='emphasis'>east</span>", "If there is no such hotel, how about one that has <span class='emphasis'>free wifi</span>", "Make sure you get <span class='emphasis'>postcode</span>, <span class='emphasis'>address</span>, and <span class='emphasis'>hotel type</span>", "You are also looking for a <span class='emphasis'>restaurant</span>. The restaurant should serve <span class='emphasis'>venetian</span> food and should be <span class='emphasis'>in

In [16]:
s= "MUL1362.json0"
print (s[:-5])

MUL1362.


## NER then write into files  

In [101]:
new_sentence=""
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    ### create a file for each sentence in both directories 
    result1= open("Data/chatito/"+object_1[:-5]+"_"+str(i)+".txt","w+")
    result2= open("Data/chatette/"+object_1[:-5]+"_"+str(i)+".txt","w+")
    ### print the label
    #print("Verbs:", [token.lemma_ for token in sentence if token.pos_ == "VERB"])
    r1 = open("Data/chatito/"+object_1[:-5]+"_"+str(i)+".txt", "w+")  
    r2 = open("Data/chatette/"+object_1[:-5]+"_"+str(i)+".txt", "w+")

    for X in (sentence.ents) :
        print([(X.text, X.label_) ])
        for z in range(len(sentence)):
            if str(sentence[z])==str(X.text):
                new_sentence+=" ["+X.label_+"] "
            else:
                new_sentence+=" "+str(sentence[z])
        
            #using chatette format
        if X.label_ in r2.read():
            r2.write("   |"+X.text)
        else:
            r2.write("~["+X.label_+"]\n")
            r2.write("   |"+X.text)

             #using chatito format
        if X.label_ in r1.read():
            r1.write("   "+X.text+"\n")
        else: 
            r1.write("~["+X.label_+"]\n")
            r1.write("   "+X.text+"\n")
    
    r1.write(str(new_sentence)+"\n")
    r2.write("\n"+str(new_sentence)+"\n")
    
    
    #labels = [x.label_ for x in sentence.ents]
    #print(len(sentence.ents))
    ### print the labels in each sentence
    #print(Counter(labels))
    ### highlight named entities and their labels in a text
    displacy.render(sentence, jupyter=True, style='ent')
    ### display the sentence and its dependencies 
    #displacy.render(sentence, style='dep', jupyter = True, options = {'distance': 120})
    
    ###apply part-of-speech tagging and lemmatize the sentence.
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        print ((x.orth_,x.pos_, x.lemma_))
    print ("******************************************************")
    

0-I'm looking for a 4 star hotel. 
[('4', 'CARDINAL')]


('I', 'PRON', '-PRON-')
("'m", 'AUX', 'be')
('looking', 'VERB', 'look')
('for', 'ADP', 'for')
('a', 'DET', 'a')
('4', 'NUM', '4')
('star', 'NOUN', 'star')
('hotel', 'NOUN', 'hotel')
******************************************************
1-I've found 21 4 star hotels. Is there a certain area you want to stay in?
[('21', 'CARDINAL')]
[('4', 'CARDINAL')]


('I', 'PRON', '-PRON-')
("'ve", 'AUX', 'have')
('found', 'VERB', 'find')
('21', 'NUM', '21')
('4', 'NUM', '4')
('star', 'NOUN', 'star')
('hotels', 'NOUN', 'hotel')
('Is', 'AUX', 'be')
('there', 'PRON', 'there')
('a', 'DET', 'a')
('certain', 'ADJ', 'certain')
('area', 'NOUN', 'area')
('you', 'PRON', '-PRON-')
('want', 'VERB', 'want')
('to', 'PART', 'to')
('stay', 'VERB', 'stay')
('in', 'ADV', 'in')
******************************************************
2-Yes, in the east, please. 


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


('Yes', 'INTJ', 'yes')
('in', 'ADP', 'in')
('the', 'DET', 'the')
('east', 'NOUN', 'east')
('please', 'INTJ', 'please')
******************************************************
3-I have found several hotels. Do you have any preference towards price?


...

('Okay', 'INTJ', 'okay')
('your', 'PRON', '-PRON-')
('taxi', 'NOUN', 'taxi')
('car', 'NOUN', 'car')
('type', 'NOUN', 'type')
('is', 'AUX', 'be')
('a', 'DET', 'a')
('white', 'ADJ', 'white')
('bmw', 'PROPN', 'bmw')
('and', 'CCONJ', 'and')
('the', 'DET', 'the')
('contact', 'NOUN', 'contact')
('number', 'NOUN', 'number')
('is', 'AUX', 'be')
('07967879848', 'NUM', '07967879848')
('Is', 'AUX', 'be')
('there', 'PRON', 'there')
('anything', 'PRON', 'anything')
('else', 'ADV', 'else')
('you', 'PRON', '-PRON-')
('need', 'VERB', 'need')
******************************************************
18-That is all.  Thank you


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


('That', 'DET', 'that')
('is', 'AUX', 'be')
('all', 'DET', 'all')
(' ', 'SPACE', ' ')
('Thank', 'VERB', 'thank')
('you', 'PRON', '-PRON-')
******************************************************
19-Great! Enjoy your stay!


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


('Great', 'ADJ', 'great')
('Enjoy', 'VERB', 'enjoy')
('your', 'PRON', '-PRON-')
('stay', 'NOUN', 'stay')
******************************************************
